# If you want to download datasets and trained models, go to this [link](https://drive.google.com/drive/folders/1Ltqj8wiuXxrl1p_KzrzfaJkn2EDfI-M4?usp=sharing). Download file named sentiment analysys using lstm.zip and unzip it

In [1]:
%config IPCompleter.use_jedi = False

In [2]:
import pandas as pd
from bs4 import BeautifulSoup
import re
import gensim
from gensim import corpora
import matplotlib.pyplot as plt
import torch
import numpy as np
%run ../Utils.ipynb

In [3]:
df = pd.read_csv(r"D:\IMDB Dataset.csv")
pd.set_option('display.max_colwidth', None)
review = df.review
df.head()

,review,sentiment
0,"One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />I would say the main appeal of the show is due to the fact that it goes where other shows wouldn't dare. Forget pretty pictures painted for mainstream audiences, forget charm, forget romance...OZ doesn't mess around. The first episode I ever saw struck me as so nasty it was surreal, I couldn't say I was ready for it, but as I watched more, I developed a taste for Oz, and got accustomed to the high levels of graphic violence. Not just violence, but injustice (crooked guards who'll be sold out for a nickel, inmates who'll kill on order and get away with it, well mannered, middle class inmates being turned into prison bitches due to their lack of street skills or prison experience) Watching Oz, you may become comfortable with what is uncomfortable viewing....thats if you can get in touch with your darker side.",positive
1,"A wonderful little production. <br /><br />The filming technique is very unassuming- very old-time-BBC fashion and gives a comforting, and sometimes discomforting, sense of realism to the entire piece. <br /><br />The actors are extremely well chosen- Michael Sheen not only ""has got all the polari"" but he has all the voices down pat too! You can truly see the seamless editing guided by the references to Williams' diary entries, not only is it well worth the watching but it is a terrificly written and performed piece. A masterful production about one of the great master's of comedy and his life. <br /><br />The realism really comes home with the little things: the fantasy of the guard which, rather than use the traditional 'dream' techniques remains solid then disappears. It plays on our knowledge and our senses, particularly with the scenes concerning Orton and Halliwell and the sets (particularly of their flat with Halliwell's murals decorating every surface) are terribly well done.",positive
2,"I thought this was a wonderful way to spend time on a too hot summer weekend, sitting in the air conditioned theater and watching a light-hearted comedy. The plot is simplistic, but the dialogue is witty and the characters are likable (even the well bread suspected serial killer). While some may be disappointed when they realize this is not Match Point 2: Risk Addiction, I thought it was proof that Woody Allen is still fully in control of the style many of us have grown to love.<br /><br />This was the most I'd laughed at one of Woody's comedies in years (dare I say a decade?). While I've never been impressed with Scarlet Johanson, in this she managed to tone down her ""sexy"" image and jumped right into a average, but spirited young woman.<br /><br />This may not be the crown jewel of his career, but it was wittier than ""Devil Wears Prada"" and more interesting than ""Superman"" a great comedy to go see with friends.",positive
3,"Basically there's a family where a little boy (Jake) thinks there's a zombie in his closet & his parents are fighting all the time.<br /><br />This movie is slower than a soap opera... and suddenly, Jake decides to become Rambo and kill the zom

## I use functions from file Utils.ipynb which contains functions for preprocessing

In [4]:
review = list(map(remove_html_tag, review))

In [5]:
review = list(map(remove_accent_char, review))

In [6]:
review = list(map(expand_contractions, review))

In [7]:
review = list(map(remove_special_characters, review))

In [1]:
review = tokenizer_and_lemmatizer(review)

In [12]:
review = list(map(lambda text: text.lower(), review))

In [13]:
df.review =review

## Save and load review text after preprocessing

In [71]:
# df.to_csv(r"C:\Users\Hp\Downloads\NLP learning path\Deep Learning Models for NLP\idbm review.csv", index=False)

In [70]:
df = pd.read_csv(r"C:\Users\Hp\Downloads\NLP learning path\Deep Learning Models for NLP\idbm review.csv")
review = df.review

In [72]:
review = list(map(tokenizer, review))

## I use stopword in nltk, but some of them have impact in sentiment analysis so i remove them from stopword list

In [73]:
new_stopwords = set(stopwords.words('english')) - {'not', 'such', 'up', 'down', 'too', 'against', 'only'}

In [74]:
def remove_stopword(text):
    text = [word for word in text if word not in new_stopwords]
    return text
review = list(map(remove_stopword, review))

## Create word2index dictionary for encoding sentences

In [75]:
word2index = corpora.Dictionary(review)

In [9]:
def encode_sent(sent):
    sent = [word2index.token2id[word] for word in sent if word in word2index.token2id.keys()]
    return sent
encoded_review = list(map(encode_sent, review))

In [10]:
review_len = [len(sent) for sent in encoded_review]

In [11]:
pd.Series(review_len).describe()

count    50000.000000
mean       122.378980
std         92.209613
min          3.000000
25%         66.000000
50%         91.000000
75%        149.000000
max       1474.000000
dtype: float64

In [12]:
count = 0
for i in review_len:
    if i >= 300: 
        count = count + 1
print(count)

2892


## LSTM need all encoded sentences have same length, so i if a sentence has its length smaller than pre-defined length, we insert 0 into start of sentence

In [13]:
def pad_features(reviews_int, seq_length):
    features = np.zeros((len(reviews_int), seq_length), dtype = int)
    
    for i, review in enumerate(reviews_int):
        review_len = len(review)
        if review_len <= seq_length:
            zeroes = list(np.zeros(seq_length-review_len))
            new = zeroes+review
        elif review_len > seq_length:
            new = review[0:seq_length]
        features[i,:] = np.array(new)
    
    return features
pad_encoded_review = pad_features(encoded_review, 300)

In [14]:
target = []
for i in df.sentiment:
    if i == 'positive':
        target.append(1)
    else:
        target.append(0)

In [15]:
X = pad_encoded_review
y = np.array(target)

## Train test split and batch loader

In [16]:
rate = 0.1/0.9
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=1)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=rate, random_state=1) # 0.25 x 0.8 = 0.2

In [17]:
from torch.utils.data import DataLoader, TensorDataset
train_data = TensorDataset(torch.from_numpy(X_train), torch.from_numpy(y_train))
valid_data = TensorDataset(torch.from_numpy(X_val), torch.from_numpy(y_val))
test_data = TensorDataset(torch.from_numpy(X_test), torch.from_numpy(y_test))

In [18]:
# dataloaders
batch_size = 20
# make sure to SHUFFLE your data
train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
valid_loader = DataLoader(valid_data, shuffle=True, batch_size=batch_size)
test_loader = DataLoader(test_data, shuffle=True, batch_size=batch_size)

In [19]:
dataiter = iter(train_loader)
sample_x, sample_y = dataiter.next()
print('Sample input size: ', sample_x.size()) # batch_size, seq_length
print('Sample input: \n', sample_x)
print()
print('Sample label size: ', sample_y.size()) # batch_size
print('Sample label: \n', sample_y)

Sample input size:  torch.Size([20, 300])
Sample input: 
 tensor([[    0,     0,     0,  ...,  1157,   751,  2092],
        [    0,     0,     0,  ...,   694,  3041,   933],
        [    0,     0,     0,  ...,  8879, 34488,  3785],
        ...,
        [    0,     0,     0,  ...,  1271,   922,  1736],
        [    0,     0,     0,  ...,   643, 20950,  2977],
        [    0,     0,     0,  ...,   285,  1047,  3713]], dtype=torch.int32)

Sample label size:  torch.Size([20])
Sample label: 
 tensor([1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1],
       dtype=torch.int32)


# Create LSTM Model

In [1]:
import torch.nn as nn
class LSTM(nn.Module):
    def __init__(self, input_size, output_size, num_layers, embedding_dim, hidden_dim, p_drop = 0.5):
        super(LSTM, self).__init__()
        self.n_layers = num_layers
        self.input_size = input_size
        self.output_size = output_size
        self.hidden_dim = hidden_dim
        
        self.embedding = nn.Embedding(input_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers, dropout = p_drop, batch_first=True)
        
        self.dropout = nn.Dropout(0.5)
        self.fc1 = nn.Linear(hidden_dim, 64)
        self.fc2 = nn.Linear(64, output_size)
        self.sigmoid = nn.Sigmoid()
    
    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        if (torch.cuda.is_available()):
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_(),
                      weight.new(self.n_layers, batch_size, self.hidden_dim).zero_())
        
        return hidden
        
    def forward(self, x, hidden):
        # x size: [batch_size, seq_len]
        batch_size=x.size()
        out = self.embedding(x)
        # out: [batch_size, seq_len, embedding_dim]
        out, hidden = self.lstm(out, hidden)
        # out: [batch_size, seq_len, hidden_dim]
        out=self.dropout(out)
        out=self.fc1(out)
        out=self.dropout(out)
        out=self.fc2(out)
        out=self.dropout(out)
        sig_out=self.sigmoid(out)
        
        sig_out=sig_out.view(batch_size, -1)
        sig_out=sig_out[:, -1]
        return sig_out, hidden
        

In [6]:
input_size = len(word2index) + 1
output_size = 1
num_layer = 2
embedding_dim = 400
hidden_layer =256

if(torch.cuda.is_available()):
    lstm_model = LSTM(input_size, output_size, num_layer, embedding_dim, hidden_layer).cuda()
else:
    lstm_model = LSTM(input_size, output_size, num_layer, embedding_dim, hidden_layer)


In [23]:
lr=0.001
epochs = 3
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(lstm_model.parameters(), lr=lr)
clip=5
print_every = 100
lstm_model.train()

LSTM(
  (embedding): Embedding(93270, 400)
  (lstm): LSTM(400, 256, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc1): Linear(in_features=256, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)

In [24]:
counter = 0
for epoch in range(epochs):
    hidden = lstm_model.init_hidden(batch_size)
    
    for inputs, labels in train_loader:
        counter += 1
        if(torch.cuda.is_available()):
            inputs=inputs.cuda()
            labels=labels.cuda()
        
        hidden = tuple([each.data for each in hidden])
        lstm_model.zero_grad()
        output, hidden = lstm_model(inputs, hidden)
        
        loss = criterion(output, labels.float())
        loss.backward()
        
        # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
        nn.utils.clip_grad_norm_(lstm_model.parameters(), clip)
        optimizer.step()
        
        if counter % print_every == 0:
            # Get validation loss
            val_h = lstm_model.init_hidden(batch_size)
            val_losses = []
            lstm_model.eval()
            for inputs, labels in valid_loader:

                # Creating new variables for the hidden state, otherwise
                # we'd backprop through the entire training history
                val_h = tuple([each.data for each in val_h])

                if(torch.cuda.is_available()):
                    inputs, labels = inputs.cuda(), labels.cuda()

                output, val_h = lstm_model(inputs, val_h)
                val_loss = criterion(output.squeeze(), labels.float())

                val_losses.append(val_loss.item())

            lstm_model.train()
            print("Epoch: {}/{}...".format(epoch+1, epochs),
                  "Step: {}...".format(counter),
                  "Loss: {:.6f}...".format(loss.item()),
                  "Val Loss: {:.6f}".format(np.mean(val_losses)))

Epoch: 1/3... Step: 100... Loss: 0.682755... Val Loss: 0.654349
Epoch: 1/3... Step: 200... Loss: 0.729988... Val Loss: 0.616379
Epoch: 1/3... Step: 300... Loss: 0.670362... Val Loss: 0.614777
Epoch: 1/3... Step: 400... Loss: 0.609532... Val Loss: 0.597646
Epoch: 1/3... Step: 500... Loss: 0.678148... Val Loss: 0.613605
Epoch: 1/3... Step: 600... Loss: 0.726528... Val Loss: 0.615318
Epoch: 1/3... Step: 700... Loss: 0.521336... Val Loss: 0.580223
Epoch: 1/3... Step: 800... Loss: 0.699753... Val Loss: 0.545811
Epoch: 1/3... Step: 900... Loss: 0.687463... Val Loss: 0.549272
Epoch: 1/3... Step: 1000... Loss: 0.697662... Val Loss: 0.688805
Epoch: 1/3... Step: 1100... Loss: 0.733532... Val Loss: 0.584191
Epoch: 1/3... Step: 1200... Loss: 0.588326... Val Loss: 0.557263
Epoch: 1/3... Step: 1300... Loss: 0.573378... Val Loss: 0.528938
Epoch: 1/3... Step: 1400... Loss: 0.708354... Val Loss: 0.478049
Epoch: 1/3... Step: 1500... Loss: 0.561122... Val Loss: 0.498979
Epoch: 1/3... Step: 1600... Loss: 

## Save model after trainning

In [37]:
PATH = "lstm_state_dict_model.pt"
torch.save(lstm_model.state_dict(), PATH)

In [7]:
if(torch.cuda.is_available()):
    load_model = LSTM(input_size, output_size, num_layer, embedding_dim, hidden_layer)
else:
    load_model = LSTM(input_size, output_size, num_layer, embedding_dim, hidden_layer)

In [39]:
load_model.load_state_dict(torch.load(PATH))
load_model.eval()

LSTM(
  (embedding): Embedding(93270, 400)
  (lstm): LSTM(400, 256, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc1): Linear(in_features=256, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)

In [40]:
ENTIRE_PATH = "entire_model.pt"
torch.save(lstm_model, ENTIRE_PATH)

## Test model

In [46]:
test_losses = [] # track loss
num_correct = 0
# init hidden state
hidden = lstm_model.init_hidden(batch_size)

lstm_model.eval()
# iterate over test data
for inputs, labels in test_loader:

    # Creating new variables for the hidden state, otherwise
    # we'd backprop through the entire training history
    hidden = tuple([each.data for each in hidden])

    if(torch.cuda.is_available()):
        inputs, labels = inputs.cuda(), labels.cuda()
    
    # get predicted outputs
    output, h = lstm_model(inputs, hidden)
    
    # calculate loss
    test_loss = criterion(output, labels.float())
    test_losses.append(test_loss.item())
    
    # convert output probabilities to predicted class (0 or 1)
    pred = torch.round(output.squeeze())  # rounds to the nearest integer
    
    # compare predictions to true label
    correct_tensor = pred.eq(labels.float().view_as(pred))
    correct = np.squeeze(correct_tensor.numpy()) if not torch.cuda.is_available() else np.squeeze(correct_tensor.cpu().numpy())
    num_correct += np.sum(correct)


# -- stats! -- ##
# avg test loss
print("Test loss: {:.3f}".format(np.mean(test_losses)))

# accuracy over all test data
test_acc = num_correct/len(test_loader.dataset)
print("Test accuracy: {:.3f}".format(test_acc))

Test loss: 0.323
Test accuracy: 0.878


## Test when put unseen data into model

In [170]:

def remove_stopword(text):
    text = [word for word in text if word not in new_stopwords]
    return text

def pad_features(reviews_int, seq_length):
    features = np.zeros((len(reviews_int), seq_length), dtype = int)

    for i, review in enumerate(reviews_int):
        review_len = len(review)
        if review_len <= seq_length:
            zeroes = list(np.zeros(seq_length-review_len))
            new = zeroes+review
        elif review_len > seq_length:
            new = review[0:seq_length]
        features[i,:] = np.array(new)

    return features

def encode_sent(sent):
    sent = [word2index.token2id[word] for word in sent if word in word2index.token2id.keys()]
    return sent

def preprocessing_input(sents):
    sents = list(map(remove_html_tag, sents))
    sents = list(map(remove_accent_char, sents))
    sents = list(map(expand_contractions, sents))
    sents = list(map(remove_special_characters, sents))
    sents = tokenizer_and_lemmatizer(sents)
    sents = list(map(tokenizer, sents))
    sents = list(map(remove_stopword, sents))
    sents = list(map(encode_sent, sents))
    sents = pad_features(sents, 300)
    return sents

def input_prepare(sents): 
    input = preprocessing_input(sents)
    input = np.array(input)
    input = torch.from_numpy(input)
    if torch.cuda.is_available():
        input = input.cuda()
    h = lstm_model.init_hidden(input.size(0))
    return input, h

In [181]:
text_input = ["it's a bad movie out of the time","you're a murder, but you still have morality"]
def make_predict(input, h):
    output, h = lstm_model(input, h)
    pred = torch.round(output) 
    for ind, prediction in enumerate(pred):
        print(text_input[ind])
        if(prediction == 1):
            print("Positive review detected!")
        else:
            print("Negative review detected.")

In [182]:
input, h = input_prepare(text_input)
make_predict(input, h)

0
1
it's a bad movie out of the time
Negative review detected.
you're a good guy, but you're aslo a fuck boy
Positive review detected!
